In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy import signal
from scipy.integrate import simps
import numpy as np
import datetime
import time

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import learning_curve, ShuffleSplit, cross_val_score, train_test_split, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler

from mne import Epochs, pick_types, events_from_annotations, EpochsArray, create_info, read_epochs
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from mne.time_frequency import psd_array_welch, psd_array_multitaper, stft
import seaborn as sns
from tqdm import tqdm

In [2]:
# keras/tensorflow

import sys
import tensorflow.keras as keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import platform

from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization, Dropout
from keras.regularizers import l2
import keras_tuner as kt
from keras.optimizers import Adam

import nengo
from nengo.utils.filter_design import cont2discrete
import nengo_dl

from keras_lmu import LMU
import keras_spiking


print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: macOS-13.3.1-arm64-arm-64bit
Tensor Flow Version: 2.9.0
Keras Version: 2.9.0

Python 3.9.15 | packaged by conda-forge | (main, Nov 22 2022, 08:48:25) 
[Clang 14.0.6 ]
Pandas 1.5.2
Scikit-Learn 1.2.0
GPU is available


# Load Data

In [3]:
source_stft = np.load('preprocessed_data/source_stft.npy')
train_idx = np.load('preprocessed_data/train_idx.npy')
val_idx = np.load('preprocessed_data/val_idx.npy')
test_idx = np.load('preprocessed_data/test_idx.npy')
labels = np.load('preprocessed_data/labels.npy')
freqs = np.load('preprocessed_data/freq.npy')
times = np.load('preprocessed_data/time.npy')

print("source_stft.shape", source_stft.shape)
print("train_idx.shape", train_idx.shape)
print("val_idx.shape", val_idx.shape)
print("test_idx.shape", test_idx.shape)
print("labels.shape", labels.shape)
print("freqs.shape", freqs.shape)
print("times.shape", times.shape)

source_stft.shape (4657, 4, 81, 9)
train_idx.shape (3725,)
val_idx.shape (466,)
test_idx.shape (466,)
labels.shape (4657,)
freqs.shape (81,)
times.shape (9,)


In [4]:
# load CSP filtered epochs from .fif file
csp_epochs = read_epochs('preprocessed_data/source_epochs-epo.fif')
csp_epochs_data = csp_epochs.get_data()
print("csp_epochs_data.shape", csp_epochs_data.shape)

Reading /Users/anushmutyala/Documents/GitHub/Energy-Efficient-Decoding-of-EEG-Motor-Imagery-using-Spiking-Legendre-Memory-Units/ml/preprocessed_data/source_epochs-epo.fif ...
Isotrak not found
    Found the data of interest:
        t =   -1000.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
4657 matching events found
No baseline correction applied
0 projection items activated
csp_epochs_data.shape (4657, 4, 801)


In [5]:
X_train = csp_epochs_data[train_idx]
X_val = csp_epochs_data[val_idx]
X_test = csp_epochs_data[test_idx]

y_train = labels[train_idx]
y_val = labels[val_idx]
y_test = labels[test_idx]

In [6]:
scaler = StandardScaler()

# move time axis to position 1 from 2 (keras rnn format)
X_train = np.moveaxis(X_train, 2, 1)
# print("X_train.shape", X_train.shape)
X_train = scaler.fit_transform(X_train.reshape((X_train.shape[0]*X_train.shape[1],X_train.shape[2]))).reshape((X_train.shape[0],X_train.shape[1],X_train.shape[2]))
print(X_train.shape)

X_val = np.moveaxis(X_val, 2, 1)
X_val = scaler.transform(X_val.reshape((X_val.shape[0]*X_val.shape[1],X_val.shape[2]))).reshape((X_val.shape[0],X_val.shape[1],X_val.shape[2]))
print(X_val.shape)

X_test = np.moveaxis(X_test, 2, 1)
X_test = scaler.transform(X_test.reshape((X_test.shape[0]*X_test.shape[1],X_test.shape[2]))).reshape((X_test.shape[0],X_test.shape[1],X_test.shape[2]))
print(X_test.shape)

(3725, 801, 4)
(466, 801, 4)
(466, 801, 4)


# Using Keras LMU

In [9]:
model = Sequential()
model.add(LMU(
    # hidden_cell=tf.keras.layers.SimpleRNNCell(units=64),
    hidden_cell=None, # removes non-linear component, atp only mapping from input to memory dimension is trained 
    memory_d=1, # tune this -> dimensionality of input after projection to memory dimension
    order=32, # number of coefficients used to represent the memory vector 
    theta=256, # sliding window that can also be tuned
    input_shape=(X_train.shape[1], X_train.shape[2]),
    # trainable_theta=True,
    ))
model.add(Dense(16, activation='relu')) # if not using hidden_cell
model.add(Dense(1, activation=tf.nn.sigmoid))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lmu_1 (LMU)                 (None, 32)                4         
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 549
Trainable params: 549
Non-trainable params: 0
_________________________________________________________________


In [17]:
history_lmu = model.fit(X_train, y_train, epochs=30, batch_size=10, validation_data=(X_val, y_val))

Epoch 1/30
373/373 [==============================] - 104s 254ms/step - loss: 0.7130 - accuracy: 0.5184 - val_loss: 0.6993 - val_accuracy: 0.5322
Epoch 2/30
373/373 [==============================] - 84s 225ms/step - loss: 0.6977 - accuracy: 0.5399 - val_loss: 0.7001 - val_accuracy: 0.5343
Epoch 3/30
373/373 [==============================] - 81s 218ms/step - loss: 0.6898 - accuracy: 0.5399 - val_loss: 0.6919 - val_accuracy: 0.5215
Epoch 4/30
373/373 [==============================] - 81s 218ms/step - loss: 0.6897 - accuracy: 0.5482 - val_loss: 0.6957 - val_accuracy: 0.5215
Epoch 5/30
373/373 [==============================] - 85s 227ms/step - loss: 0.6852 - accuracy: 0.5514 - val_loss: 0.6970 - val_accuracy: 0.5021
Epoch 6/30
282/373 [=====================>........] - ETA: 21s - loss: 0.6832 - accuracy: 0.5482

KeyboardInterrupt: 

# LSTM

In [15]:
model = Sequential()
model.add(LSTM(32, input_shape=(X_train.shape[1], X_train.shape[2]), dropout=0.1))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01)))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 32)                4736      
                                                                 
 batch_normalization_2 (Batc  (None, 32)               128       
 hNormalization)                                                 
                                                                 
 dense_6 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4,897
Trainable params: 4,833
Non-trainable params: 64
_________________________________________________________________


In [16]:
history_lstm = model.fit(X_train, y_train, batch_size=128, epochs=30, validation_data=(X_val, y_val))

Epoch 1/30
30/30 [==============================] - 30s 703ms/step - loss: 0.7446 - accuracy: 0.5015 - val_loss: 0.7091 - val_accuracy: 0.5107
Epoch 2/30
30/30 [==============================] - 8s 256ms/step - loss: 0.7144 - accuracy: 0.5050 - val_loss: 0.7081 - val_accuracy: 0.5172
Epoch 3/30
30/30 [==============================] - 8s 259ms/step - loss: 0.7088 - accuracy: 0.5187 - val_loss: 0.7074 - val_accuracy: 0.5236
Epoch 4/30
30/30 [==============================] - 8s 254ms/step - loss: 0.7054 - accuracy: 0.5187 - val_loss: 0.7060 - val_accuracy: 0.5236
Epoch 5/30
30/30 [==============================] - 8s 253ms/step - loss: 0.7043 - accuracy: 0.5152 - val_loss: 0.7055 - val_accuracy: 0.5494
Epoch 6/30
30/30 [==============================] - 9s 311ms/step - loss: 0.7030 - accuracy: 0.5246 - val_loss: 0.7046 - val_accuracy: 0.5451
Epoch 7/30
30/30 [==============================] - 8s 271ms/step - loss: 0.7014 - accuracy: 0.5227 - val_loss: 0.7037 - val_accuracy: 0.5279
Epoch

KeyboardInterrupt: 